In [1]:
import os
import pprint
import tempfile
import urllib

import numpy as np
import pandas as pd

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import trainer_pb2
from tfx.proto import pusher_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from tfx.utils.dsl_utils import external_input


%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

_serving_model_dir = os.path.join(os.getcwd(), 'serving_model')
_context_dir = os.path.join(os.getcwd(), 'context')
_data_dir = os.path.join(os.getcwd(), '../../data/titanic')
_data_filepath = os.path.join(_data_dir, "data.csv")

absl.logging.set_verbosity(absl.logging.INFO)

context = InteractiveContext(pipeline_root=_context_dir)

# ExampleGen

In [2]:
example_gen = CsvExampleGen(input=external_input(_data_dir))
context.run(example_gen)

Instructions for updating:
external_input is deprecated, directly pass the uri to ExampleGen.
INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.


INFO:absl:Processing input csv data /Users/rkazinnik/PycharmProjects/jim/azumo/kazinnik_mlops/tfx_local/titanic/../../data/titanic/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 94
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 108
        type_id: 5
        uri: "/Users/rkazinnik/PycharmProjects/jim/azumo/kazinnik_mlops/tfx_local/titanic/context/CsvExampleGen/examples/94"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:103548,xor_checksum:1611861413,sum_checksum:1611861413"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            string_value: "0"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        ))

In [3]:
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'train')
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "age"
    value {
      float_list {
        value: 0.916700005531311
      }
    }
  }
  feature {
    key: "boat"
    value {
      bytes_list {
        value: "11"
      }
    }
  }
  feature {
    key: "body"
    value {
      int64_list {
      }
    }
  }
  feature {
    key: "cabin"
    value {
      bytes_list {
        value: "C22 C26"
      }
    }
  }
  feature {
    key: "embarked"
    value {
      bytes_list {
        value: "S"
      }
    }
  }
  feature {
    key: "fare"
    value {
      float_list {
        value: 151.5500030517578
      }
    }
  }
  feature {
    key: "home.dest"
    value {
      bytes_list {
        value: "Montreal  PQ   Chesterville  ON"
      }
    }
  }
  feature {
    key: "name"
    value {
      bytes_list {
        value: "Allison  Master. Hudson Trevor"
      }
    }
  }
  feature {
    key: "parch"
    value {
      int64_list {
        value: 2
      }
    }
  }
  feature {
    key: "pclass"
    value {


# StatisticsGen

In [4]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)
context.show(statistics_gen.outputs['statistics'])

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to /Users/rkazinnik/PycharmProjects/jim/azumo/kazinnik_mlops/tfx_local/titanic/context/StatisticsGen/statistics/95/train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to /Users/rkazinnik/PycharmProjects/jim/azumo/kazinnik_mlops/tfx_local/titanic/context/StatisticsGen/statistics/95/eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [5]:
example_gen = CsvExampleGen(input=external_input(_data_dir))
context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 96
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 108
        type_id: 5
        uri: "/Users/rkazinnik/PycharmProjects/jim/azumo/kazinnik_mlops/tfx_local/titanic/context/CsvExampleGen/examples/94"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:103548,xor_checksum:1611861413,sum_checksum:1611861413"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            string_value: "0"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        create_time_since_epoch: 1614631273290
        last_update_time_since_epoch: 1614631279936
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        ))

# SchemaGen

In [6]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen)
context.show(schema_gen.outputs['schema'])

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to /Users/rkazinnik/PycharmProjects/jim/azumo/kazinnik_mlops/tfx_local/titanic/context/SchemaGen/schema/97/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


,Type,Presence,Valency,Domain
Feature name,,,,
'age',FLOAT,required,,-
'boat',STRING,required,,'boat'
'body',INT,required,,-
'cabin',BYTES,required,,-
'embarked',STRING,required,,'embarked'
'fare',FLOAT,required,single,-
'home.dest',BYTES,required,,-
'name',BYTES,required,single,-
'parch',INT,required,single,-


/Users/rkazinnik/anaconda3/envs/tfx003/lib/python3.6/site-packages/tensorflow_data_validation/utils/display_util.py:151: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'boat',"'1', '10', '11', '12', '13', '13 15', '13 15 B', '14', '15', '15 16', '16', '2', '3', '4', '5', '5 7', '6', '7', '8', '8 10', '9', 'A', 'B', 'C', 'C D', 'D', '5 9'"
'embarked',"'C', 'Q', 'S'"
'sex',"'female', 'male'"


# ExampleValidator

In [7]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)
context.show(example_validator.outputs['anomalies'])

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to /Users/rkazinnik/PycharmProjects/jim/azumo/kazinnik_mlops/tfx_local/titanic/context/ExampleValidator/anomalies/98/train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to /Users/rkazinnik/PycharmProjects/jim/azumo/kazinnik_mlops/tfx_local/titanic/context/ExampleValidator/anomalies/98/eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


# Transform

In [8]:
%%writefile "constants.py"

MAX_CATEGORICAL_FEATURE_VALUES = [3,
                                  2,
                                 ]
CATEGORICAL_FEATURE_KEYS = ['pclass',
                            'is_alone',
                           ]

DENSE_FLOAT_FEATURE_KEYS = [
                            'fare',
                            'family_size',
                            'age_pclass',
                            'name_count',
                            'cabin_count',
                            'fare_per_person'
                           ]

BUCKET_FEATURE_BUCKET_COUNT = [3,
                               10,
                               10
                              ]
BUCKET_FEATURE_KEYS = ['age',
                       'sibsp',
                       'parch'
                      ]

VOCAB_SIZE = 1000
OOV_SIZE = 10
VOCAB_FEATURE_KEYS = [
     'boat',
     'embarked',
     'sex',
     'home.dest',
     'ticket_type'
]

LABEL_KEY = 'survived'

def transformed_name(key):
  return key + '_xf'

Overwriting constants.py


In [9]:
%%writefile "transform.py"

import tensorflow as tf
import tensorflow_transform as tft

import constants

_DENSE_FLOAT_FEATURE_KEYS = constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = constants.VOCAB_FEATURE_KEYS
_VOCAB_SIZE = constants.VOCAB_SIZE
_OOV_SIZE = constants.OOV_SIZE
_BUCKET_FEATURE_KEYS = constants.BUCKET_FEATURE_KEYS
_BUCKET_FEATURE_BUCKET_COUNT = constants.BUCKET_FEATURE_BUCKET_COUNT
_CATEGORICAL_FEATURE_KEYS = constants.CATEGORICAL_FEATURE_KEYS
_LABEL_KEY = constants.LABEL_KEY
_transformed_name = constants.transformed_name

def preprocessing_fn(inputs):
  # we must use tf or tft only
  # substr works with dense tensor only 

  inputs["ticket_type"] = tf.strings.substr(_fill_in_missing(inputs['ticket']),0,3)
  
  inputs["family_size"] = _fill_in_missing(inputs['sibsp']) + _fill_in_missing(inputs['parch']) + 1

  inputs["is_alone"] = tf.cast(tf.math.equal(inputs['family_size'],tf.constant(1,dtype=tf.int64)),tf.int64)

  inputs['age_pclass'] = tf.math.multiply(tf.cast(_fill_in_missing(inputs['age']),tf.float32), tf.cast(_fill_in_missing(inputs['pclass']),tf.float32))
    
  inputs['name_count'] = tft.word_count(tf.strings.split(_fill_in_missing(inputs["name"]),sep=" ")) -1 

  inputs['cabin_count'] = tft.word_count(tf.strings.split(_fill_in_missing(inputs["cabin"]),sep=" ")) -1 

  inputs['fare_per_person']= tf.math.multiply(tf.cast(_fill_in_missing(inputs['fare']),tf.float32), tf.cast(_fill_in_missing(inputs['family_size']),tf.float32))
    
  outputs = {}
    
  for key in _DENSE_FLOAT_FEATURE_KEYS:
    outputs[_transformed_name(key)] = tft.scale_to_z_score(
        _fill_in_missing(inputs[key]))

  for key in _VOCAB_FEATURE_KEYS:
    outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(
            _fill_in_missing(inputs[key]),
            top_k=_VOCAB_SIZE,
            num_oov_buckets=_OOV_SIZE)

  for key, num_buckets in zip(_BUCKET_FEATURE_KEYS,
                              _BUCKET_FEATURE_BUCKET_COUNT):
    outputs[_transformed_name(key)] = tft.bucketize(
        _fill_in_missing(inputs[key]), num_buckets)

  for key in _CATEGORICAL_FEATURE_KEYS:
    outputs[_transformed_name(key)] = _fill_in_missing(inputs[key])       
    
  outputs[_transformed_name(_LABEL_KEY)] = _fill_in_missing(inputs[_LABEL_KEY])

  return outputs


def _fill_in_missing(x):
  if not isinstance(x,tf.sparse.SparseTensor):
    return x

  default_value = '' if x.dtype == tf.string else 0
  return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value),
      axis=1)

Overwriting transform.py


# Transform

In [10]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file="transform.py")
context.run(transform)

INFO:absl:Running driver for Transform
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Transform
INFO:absl:Analyze the 'train' split and transform all splits when splits_config is not set.


Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`


INFO:absl:Loading transform.py because it has not been loaded before.
INFO:absl:transform.py is already loaded.
INFO:absl:Feature age has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature boat has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature body has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature cabin has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature embarked has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature fare has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature home.dest has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature name has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature parch has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature pclass has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature sex has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature sibsp has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature survived has no shape. Setting to VarLenSp

Instructions for updating:
Use ref() instead.


INFO:absl:Feature age has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature boat has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature body has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature cabin has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature embarked has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature fare has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature home.dest has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature name has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature parch has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature pclass has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature sex has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature sibsp has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature survived has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature ticket has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature age has no s

...

INFO:absl:Feature age has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature boat has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature body has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature cabin has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature embarked has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature fare has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature home.dest has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature name has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature parch has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature pclass has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature sex has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature sibsp has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature survived has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature ticket has no shape. Setting to VarLenSparseTensor.


INFO:absl:Feature age has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature boat has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature body has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature cabin has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature embarked has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature fare has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature home.dest has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature name has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature parch has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature pclass has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature sex has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature sibsp has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature survived has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature ticket has no shape. Setting to VarLenSparseTensor.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:Assets written to: /Users/rkazinnik/PycharmProjects/jim/azumo/kazinnik_mlops/tfx_local/titanic/context/Transform/transform_graph/99/.temp_path/tftransform_tmp/6a011c53adbc4b7fac93d990f0d093ce/assets
INFO:tensorflow:SavedModel written to: /Users/rkazinnik/PycharmProjects/jim/azumo/kazinnik_mlops/tfx_local/titanic/context/Transform/transform_graph/99/.temp_path/tftransform_tmp/6a011c53adbc4b7fac93d990f0d093ce/saved_model.pb
value: "\n\013\n\tConst_2:0\022-vocab_compute_and_apply_vocabulary_vocabulary"

value: "\n\013\n\tConst_4:0\022/vocab_compute_and_apply_vocabulary_1_vocabulary"

value: "\n\013\n\tConst_6:0\022/vocab_compute_and_apply_vocabulary_2_vocabulary"

value: "\n\013\n\tConst_8:0\022/vocab_compute_and_apply_vocabulary_3_vocabulary"

value: "\n\014\

INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 99
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 112
        type_id: 13
        uri: "/Users/rkazinnik/PycharmProjects/jim/azumo/kazinnik_mlops/tfx_local/titanic/context/Transform/transform_graph/99"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 13
        name: "TransformGraph"
        )]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 113
        type_id: 5
        uri: "/Users/rkazinnik/PycharmProjects/jim/azumo/kazinnik_mlops/tfx_local/titanic/context/Transform/transformed_examples/99"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 114
        type_id: 14
        uri: "/Users/rkazinnik/PycharmProjects/jim/azumo/kazinnik_mlops/tfx_local/titanic/context/Transform/updated_analyzer_cache/99"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "TransformCache"
        )]
        ))

# Print transformed data 

In [15]:
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'train')

tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

for tfrecord in dataset.take(1):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "age_pclass_xf"
    value {
      float_list {
        value: -1.250217318534851
      }
    }
  }
  feature {
    key: "age_xf"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "boat_xf"
    value {
      int64_list {
        value: 8
      }
    }
  }
  feature {
    key: "cabin_count_xf"
    value {
      float_list {
        value: 3.685793876647949
      }
    }
  }
  feature {
    key: "embarked_xf"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "family_size_xf"
    value {
      float_list {
        value: 1.396721363067627
      }
    }
  }
  feature {
    key: "fare_per_person_xf"
    value {
      float_list {
        value: 3.09846830368042
      }
    }
  }
  feature {
    key: "fare_xf"
    value {
      float_list {
        value: 2.2105491161346436
      }
    }
  }
  feature {
    key: "home.dest_xf"
    value {
      int64_list {
        value: 16
      }
    }

# Trainer

In [16]:
%%writefile "trainer.py"

from typing import List, Text

import os
import absl
import datetime
import tensorflow as tf
import tensorflow_transform as tft

from tfx.components.trainer.executor import TrainerFnArgs
from tfx.components.trainer.fn_args_utils import DataAccessor
from tfx_bsl.tfxio import dataset_options

import constants

_DENSE_FLOAT_FEATURE_KEYS = constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = constants.VOCAB_FEATURE_KEYS
_VOCAB_SIZE = constants.VOCAB_SIZE
_OOV_SIZE = constants.OOV_SIZE
_BUCKET_FEATURE_BUCKET_COUNT = constants.BUCKET_FEATURE_BUCKET_COUNT
_BUCKET_FEATURE_KEYS = constants.BUCKET_FEATURE_KEYS
_CATEGORICAL_FEATURE_KEYS = constants.CATEGORICAL_FEATURE_KEYS
_MAX_CATEGORICAL_FEATURE_VALUES = constants.MAX_CATEGORICAL_FEATURE_VALUES
_LABEL_KEY = constants.LABEL_KEY
_transformed_name = constants.transformed_name


def _transformed_names(keys):
  return [_transformed_name(key) for key in keys]


def _get_serve_tf_examples_fn(model, tf_transform_output):
  """Returns a function that parses a serialized tf.Example and applies TFT."""

  model.tft_layer = tf_transform_output.transform_features_layer()

  @tf.function
  def serve_tf_examples_fn(serialized_tf_examples):
    """Returns the output to be used in the serving signature."""
    feature_spec = tf_transform_output.raw_feature_spec()
    feature_spec.pop(_LABEL_KEY)
    parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
    transformed_features = model.tft_layer(parsed_features)
    return model(transformed_features)

  return serve_tf_examples_fn


def _input_fn(file_pattern: List[Text],
              data_accessor: DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:

  return data_accessor.tf_dataset_factory(
      file_pattern,
      dataset_options.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_transformed_name(_LABEL_KEY)),
      tf_transform_output.transformed_metadata.schema)


def _build_keras_model(hidden_units: List[int] = None) -> tf.keras.Model:

  real_valued_columns = [
      tf.feature_column.numeric_column(key, shape=())
      for key in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS)
  ]
  categorical_columns = [
      tf.feature_column.categorical_column_with_identity(
          key, num_buckets=_VOCAB_SIZE + _OOV_SIZE, default_value=0)
      for key in _transformed_names(_VOCAB_FEATURE_KEYS)
  ]
  categorical_columns += [
      tf.feature_column.categorical_column_with_identity(  # pylint: disable=g-complex-comprehension
          key,
          num_buckets=num_buckets,
          default_value=0) for key, num_buckets in zip(
              _transformed_names(_BUCKET_FEATURE_KEYS),
              _BUCKET_FEATURE_BUCKET_COUNT)
  ]
  categorical_columns += [
      tf.feature_column.categorical_column_with_identity(  # pylint: disable=g-complex-comprehension
          key,
          num_buckets=num_buckets,
          default_value=0) for key, num_buckets in zip(
              _transformed_names(_CATEGORICAL_FEATURE_KEYS),
              _MAX_CATEGORICAL_FEATURE_VALUES)
  ]

    
  indicator_column = [
      tf.feature_column.indicator_column(categorical_column)
      for categorical_column in categorical_columns
  ]

    
    


  model = _wide_and_deep_classifier(
      # TODO(b/139668410) replace with premade wide_and_deep keras model
      wide_columns=indicator_column,
      deep_columns=real_valued_columns,
      dnn_hidden_units=hidden_units or [100, 70, 50, 25])
  return model


def _wide_and_deep_classifier(wide_columns, deep_columns, dnn_hidden_units):

  input_layers = {
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype=tf.float32)
      for colname in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS)
  }
  input_layers.update({
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype='int32')
      for colname in _transformed_names(_VOCAB_FEATURE_KEYS)
  })
  input_layers.update({
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype='int32')
      for colname in _transformed_names(_BUCKET_FEATURE_KEYS)
  })
  input_layers.update({
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype='int32')
      for colname in _transformed_names(_CATEGORICAL_FEATURE_KEYS)
  }) 
    

  deep = tf.keras.layers.DenseFeatures(deep_columns)(input_layers)
  for numnodes in dnn_hidden_units:
    deep = tf.keras.layers.Dense(numnodes)(deep)
  wide = tf.keras.layers.DenseFeatures(wide_columns)(input_layers)

  output = tf.keras.layers.Dense(
      1, activation='sigmoid')(
          tf.keras.layers.concatenate([deep, wide]))

  output = tf.squeeze(output, -1)

  model = tf.keras.Model(input_layers, output)
  model.compile(
      loss='binary_crossentropy',
      optimizer=tf.keras.optimizers.Adam(lr=0.001),
      metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.AUC()])
  model.summary(print_fn=absl.logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: TrainerFnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  # Number of nodes in the first layer of the DNN
  first_dnn_layer_size = 100
  num_dnn_layers = 4
  dnn_decay_factor = 0.7

  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

  train_dataset = _input_fn(fn_args.train_files, fn_args.data_accessor, 
                            tf_transform_output, 40)
  eval_dataset = _input_fn(fn_args.eval_files, fn_args.data_accessor, 
                           tf_transform_output, 40)

  model = _build_keras_model(
      # Construct layers sizes with exponetial decay
      hidden_units=[
          max(2, int(first_dnn_layer_size * dnn_decay_factor**i))
          for i in range(num_dnn_layers)
      ])


  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=fn_args.model_run_dir, update_freq='batch')
  
  csv_logger = tf.keras.callbacks.CSVLogger(os.path.join(fn_args.model_run_dir,"metrics_train.csv"), separator=",", append=False)

  model.fit(
      train_dataset,
      epochs=4,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps,
      callbacks=[tensorboard_callback,csv_logger])


  signatures = {
      'serving_default':
          _get_serve_tf_examples_fn(model,
                                    tf_transform_output).get_concrete_function(
                                        tf.TensorSpec(
                                            shape=[None],
                                            dtype=tf.string,
                                            name='examples')),
  }
  model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Overwriting trainer.py


# Trainer

In [18]:
trainer = Trainer(
    module_file="trainer.py",
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=100), # 10000
    eval_args=trainer_pb2.EvalArgs(num_steps=50)) #5000
context.run(trainer)

INFO:absl:Running driver for Trainer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 101
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 115
        type_id: 16
        uri: "/Users/rkazinnik/PycharmProjects/jim/azumo/kazinnik_mlops/tfx_local/titanic/context/Trainer/model/100"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        create_time_since_epoch: 1614631541646
        last_update_time_since_epoch: 1614631551050
        , artifact_type: id: 16
        name: "Model"
        )]
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 116
        type_id: 17
        uri: "/Users/rkazinnik/PycharmProjects/jim/azumo/kazinnik_mlops/tfx_local/titanic/context/Trainer/model_run/100"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        create_time_since_epoch: 1614631541646
        last_update_time_since_epoch: 1614631551051
        , artifact_type: id: 17
        name: "ModelRun"
        )]
        ))

In [19]:
model_run_artifact_dir = trainer.outputs['model_run'].get()[0].uri
print("model_run_artifact_dir",model_run_artifact_dir)
%load_ext tensorboard
%tensorboard --logdir {model_run_artifact_dir}

model_run_artifact_dir /Users/rkazinnik/PycharmProjects/jim/azumo/kazinnik_mlops/tfx_local/titanic/context/Trainer/model_run/100


ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Traceback (most recent call last):
  File "/Users/rkazinnik/anaconda3/envs/tfx003/bin/tensorboard", line 8, in <module>
    sys.exit(run_main())
  File "/Users/rkazinnik/anaconda3/envs/tfx003/lib/python3.6/site-packages/tensorboard/main.py", line 46, in run_main
    app.run(tensorboard.main, flags_parser=tensorboard.configure)
  File "/Users/rkazinnik/anaconda3/envs/tfx003/lib/python3.6/site-packages/absl/app.py", line 300, in run
    _run_main(main, args)
  File "/Users/rkazinnik/anaconda3/envs/tfx003/lib/python3.6/site-packages/absl/app.py", line 251, in _run_main
    sys.exit(main(argv))
  File "/Users/rkazinnik/anaconda3/envs/tfx003/lib/python3.6/site-packages/tensorboard/program.py", line 258, in main
    return runner(self.flags) or 0
  File "/Users/rkazinnik/anaconda3/envs/tfx003/lib/python3.6/site-packages/tensorboard/program.py", line 274, in _run_serve_subcommand
    server = self._make_server()
  File "

In [20]:
df = pd.read_csv(os.path.join(model_run_artifact_dir,"metrics_train.csv"))
df.head(5)

,epoch,auc,binary_accuracy,loss,val_auc,val_binary_accuracy,val_loss
0,0,0.757734,0.7360,0.564106,0.820585,0.7330,0.529540
1,1,0.848197,0.7745,0.498970,0.868892,0.7595,0.483000
2,2,0.889847,0.8095,0.457526,0.911604,0.8000,0.449983
3,3,0.920855,0.8405,0.417770,0.933448,0.8315,0.416157


# Evaluator

In [21]:
eval_config = tfma.EvalConfig(
    model_specs=[
        # This assumes a serving model with signature 'serving_default'. If
        # using estimator based EvalSavedModel, add signature_name: 'eval' and 
        # remove the label_key.
        tfma.ModelSpec(label_key='survived')
    ],
    metrics_specs=[
        tfma.MetricsSpec(
            # The metrics added here are in addition to those saved with the
            # model (assuming either a keras model or EvalSavedModel is used).
            # Any metrics added into the saved model (for example using
            # model.compile(..., metrics=[...]), etc) will be computed
            # automatically.
            # To add validation thresholds for metrics saved with the model,
            # add them keyed by metric name to the thresholds map.
            metrics=[
                tfma.MetricConfig(class_name='BinaryAccuracy'),
                tfma.MetricConfig(class_name='AUC')
            ]
        )
    ],
    slicing_specs=[
        # An empty slice spec means the overall slice, i.e. the whole dataset.
        tfma.SlicingSpec(),
        # Data can be sliced along a feature column. In this case, data is
        # sliced along feature column trip_start_hour.
        # tfma.SlicingSpec(feature_keys=['TS_CAT_Week'])
    ])

evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    #baseline_model=model_resolver.outputs['model'],
    # Change threshold will be ignored if there is no baseline (first run).
    eval_config=eval_config)
context.run(evaluator)

INFO:absl:Running driver for Evaluator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Evaluator
        please use "has_baseline" instead.
INFO:absl:Request was made to ignore the baseline ModelSpec and any change thresholds. This is likely because a baseline model was not provided: updated_config=
model_specs {
  label_key: "survived"
}
slicing_specs {
}
metrics_specs {
  metrics {
    class_name: "BinaryAccuracy"
  }
  metrics {
    class_name: "AUC"
  }
}

INFO:absl:Using /Users/rkazinnik/PycharmProjects/jim/azumo/kazinnik_mlops/tfx_local/titanic/context/Trainer/model/100/serving_model_dir as  model.



Two checkpoint references resolved to different objects (<tensorflow.python.keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7feb924eb748> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7febb25e4908>).


INFO:absl:The 'example_splits' parameter is not set, using 'eval' split.
INFO:absl:Evaluating model.
INFO:absl:Request was made to ignore the baseline ModelSpec and any change thresholds. This is likely because a baseline model was not provided: updated_config=
model_specs {
  label_key: "survived"
}
slicing_specs {
}
metrics_specs {
  metrics {
    class_name: "BinaryAccuracy"
  }
  metrics {
    class_name: "AUC"
  }
  model_names: ""
}

INFO:absl:Request was made to ignore the baseline ModelSpec and any change thresholds. This is likely because a baseline model was not provided: updated_config=
model_specs {
  label_key: "survived"
}
slicing_specs {
}
metrics_specs {
  metrics {
    class_name: "BinaryAccuracy"
  }
  metrics {
    class_name: "AUC"
  }
  model_names: ""
}

INFO:absl:Request was made to ignore the baseline ModelSpec and any change thresholds. This is likely because a baseline model was not provided: updated_config=
model_specs {
  label_key: "survived"
}
slicing_spec


Two checkpoint references resolved to different objects (<tensorflow.python.keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7febb21b0d68> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7feb9236e240>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7febb2505be0> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7febd31aad30>).


INFO:absl:Evaluation complete. Results written to /Users/rkazinnik/PycharmProjects/jim/azumo/kazinnik_mlops/tfx_local/titanic/context/Evaluator/evaluation/102.
INFO:absl:No threshold configured, will not validate model.
INFO:absl:Running publisher for Evaluator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Evaluator
    execution_id: 102
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 117
        type_id: 19
        uri: "/Users/rkazinnik/PycharmProjects/jim/azumo/kazinnik_mlops/tfx_local/titanic/context/Evaluator/evaluation/102"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 19
        name: "ModelEvaluation"
        )]
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 118
        type_id: 20
        uri: "/Users/rkazinnik/PycharmProjects/jim/azumo/kazinnik_mlops/tfx_local/titanic/context/Evaluator/blessing/102"
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "ModelBlessing"
        )]
        ))

# tensorflow_model_analysis

In [22]:
import tensorflow_model_analysis as tfma

output_path = evaluator.outputs['evaluation'].get()[0].uri

eval_result = tfma.load_eval_result(output_path)

eval_result.slicing_metrics

[((),
  {'': {'': {'binary_accuracy': {'doubleValue': 0.8298368453979492},
     'auc': {'doubleValue': 0.9335595965385437}}}})]

In [23]:
tfma.view.render_slicing_metrics(eval_result)

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Overall', 'metrics':…

# Pusher

In [ ]:
pusher = Pusher(
    model=trainer.outputs['model'],
#    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher)
pusher.outputs
model_artifact_dir = trainer.outputs['model'].get()[0].uri
model_dir = os.path.join(model_artifact_dir, 'serving_model_dir')
model = tf.saved_model.load(model_dir)

for item in model.signatures.items():
  pp.pprint(item)

# Path to SavedModel

In [ ]:


_serving_model_dir

'/Users/rkazinnik/PycharmProjects/jim/azumo/kazinnik_mlops/tfx_local/titanic/serving_model'